In [1]:
from selenium import webdriver
import time
import pandas as pd
import re

In [2]:
def StarConversion(value):
    if not value:
        value = 0

    else:
        value = int(value)
        value = value / 100 * 5

    return round(value, 1)


def ModelExtraction(link):
    link = link[8:]
    link = link.split('/')
    return link[6][:-7]

In [3]:
# Set up the path to the chrome driver
PATH = "E:\chrome_driver_32bit\chromedriver.exe"
driver = webdriver.Chrome(PATH)

all_details = []
for page_no in range(1, 8):
    try:
        # Get current page
        driver.get("https://www.dell.com/en-uk/shop/scc/sr/laptops/inspiron-laptops?page={}".format(page_no))
        print("\nPage:\nhttps://www.dell.com/en-uk/shop/scc/sr/laptops/inspiron-laptops/page={}".format(page_no))

        # Find all laptops in the page
        laptops = driver.find_elements_by_class_name('ps-title')
        # Generate a list of links for each and every laptop
        links = []
        
        print("\nDetected Laptops:")
        for i in range(len(laptops)):
            item = laptops[i]
            # Get the href property
            link = item.find_element_by_tag_name("a").get_property("href")
            # Append the link to list links
            links.append(link)
            print(link)

        # Loop through each link to access the page of each laptop
        for l in links:
            # get one laptop url
            driver.get(l)
            time.sleep(2)

            # Model name of the laptop
            model_name = ModelExtraction(l)
            print(model_name)

            # Original price of laptop
            try:
                original_price = driver.find_element_by_xpath(
                    "/html/body/main[@class='mobile-sub-nav-enabled ']/section[@class='container mt-3 back-to-top-container']/div[@class='row hero-content']/div[@class='col-md-6 ']/div[2]/div[@class='mb-1']/div[1]/del[@class='h6']"
                )
                original_price = original_price.text
                
            except:
                original_price = None
            
            # Price of the laptop after sale
            price = driver.find_element_by_xpath(
                "/html/body/main[@class='mobile-sub-nav-enabled ']/section[@class='container mt-3 back-to-top-container']/div[@class='row hero-content']/div[@class='col-md-6 ']/div[2]/div[@class='mb-1']/span[1]/span[@class='h3 font-weight-bold mb-1 sale-price ']"
            )
            price_after_sale = price.text

            # specs:
            processor = driver.find_element_by_xpath(
                "//section[@class='container mt-3 back-to-top-container']//div[@class='row'][1]//div//ul//li[1]//p"
            )
            
            GPU = driver.find_element_by_xpath(
                "//section[@class='container mt-3 back-to-top-container']//div[@class='row'][1]//div//ul//li[3]//p"
            )
            
            memory = driver.find_element_by_xpath(
                "//section[@class='container mt-3 back-to-top-container']//div[@class='row'][1]//div//ul//li[5]//p"
            )
            
            hard_drive = driver.find_element_by_xpath(
                "//section[@class='container mt-3 back-to-top-container']//div[@class='row'][1]//div//ul//li[6]//p"
            )
            
            LCD = driver.find_element_by_xpath(
                "//section[@class='container mt-3 back-to-top-container']//div[@class='row hero-content']/div[@class='col-md-6']/div[@id='configuration']/div[@class='d-flex flex-wrap options'][6]/div[@class='btn btn-outline-dark option text-left position-relative d-flex flex-column rounded-sm']/span"
            )

            # Offer points
            offer_points = driver.find_element_by_xpath(
                "//section[@class='container mt-3 back-to-top-container']//div[@class='row hero-content']//div[@class='col-md-6 ']//div[2]//div[@class='border-bottom d-none d-md-flex flex-wrap align-items-center mb-5 pb-5']//div[@class='d-inline-block upd-rewards align-middle']//span[@class='rewards-text align-middle']")

            # Offer points as string, so we need to use this regex to extract digits
            offer_points = int(re.findall("\d+", offer_points.text)[0])

            # Stars - Rating
            stars = driver.find_element_by_xpath(
                "/html/body/main[@class='mobile-sub-nav-enabled ']/section[@class='container mt-3 back-to-top-container']/div[@class='row hero-content']/div[@class='col-md-6 ']/div[1]/div[@class='mb-4']/a[@class='rating-container ']/span").get_attribute("class")

            # Convert string percentage -in element attribute- to a number of stars.
            stars = StarConversion(stars.split('-')[-1])
            
            
            # Price
            if original_price == None:
                original_price = price.text
                price_after_sale = "Nan"
            
            # Define a dictionary with details we need
            record = {
                "Model_Name": model_name,
                "Original_Price": original_price,
                "Price_After_Sale": price_after_sale,
                "Processor": f'"{processor.text}"',
                "GPU": f'"{GPU.text}"',
                "Memory": f'"{memory.text}"',
                "Hard_drive": f'"{hard_drive.text}"',
                "LCD": f'"{LCD.text}"',
                "Offer_Points": offer_points,
                "Stars": stars
            }
            # Append record to all details
            all_details.append(record)

            '''next_button = driver.find_element_by_xpath(
                f"//html//body//main//div//section//div//div//div//div//button[@class='system-result-btn pagination-btn hide-xs'][{page_no}]").find_element_by_tag_name(
                "button").click()'''

    except Exception as e:
        # Close the browser if we run to an error
        print(e)
        driver.close()


Page:
https://www.dell.com/en-uk/shop/scc/sr/laptops/inspiron-laptops/page=1

Detected Laptops:
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15-3000/spd/inspiron-15-3510-laptop/cn31006sc11
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15-3000/spd/inspiron-15-3515-laptop/cn31511sc11
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15/spd/inspiron-15-3521-laptop/cn32106sc
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15-3000/spd/inspiron-15-3515-laptop/cn31509sc11
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15/spd/inspiron-15-3521-laptop/cn32102sc
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15-3000/spd/inspiron-15-3510-laptop/cn31005sc11
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15-3000/spd/inspiron-15-3515-laptop/cn31505sc11
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-14/spd/inspiron-14-5415-lapto

https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-16-5620/spd/inspiron-16-5620-laptop/cn56114sc
inspiron-14-5410-2-in-1
inspiron-14-7420-2-in-1
inspiron-16-5625
inspiron-15-5510
inspiron-14-5420
inspiron-16-7620-2-in-1
inspiron-14-5420
inspiron-14-7420-2-in-1
inspiron-15-5510
inspiron-16-5625
inspiron-15-7510
inspiron-16-5620

Page:
https://www.dell.com/en-uk/shop/scc/sr/laptops/inspiron-laptops/page=6

Detected Laptops:
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/new-inspiron-14-2-in-1/spd/inspiron-14-7425-2-in-1-laptop/cn74103sc
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15-plus/spd/inspiron-15-7510-laptop/cn51704sc11
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-15-plus/spd/inspiron-15-7510-laptop/bn51704sb11
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/inspiron-16-plus/spd/inspiron-16-7610-laptop/cn76108sc11
https://www.dell.com/en-uk/shop/laptop-computers-2-in-1-pcs/new-inspiron-16-2

In [4]:
df = pd.DataFrame(all_details)
df

,Model_Name,Original_Price,Price_After_Sale,Processor,GPU,Memory,Hard_drive,LCD,Offer_Points,Stars
0,inspiron-15-3510,£299.00,£269.00,"""Intel® Celeron® N4020 (4 MB cache, 2 cores, 2...","""Intel® UHD Graphics 600""","""4 GB, 1 x 4 GB, DDR4, 2400 MHz""","""128 GB, M.2, PCIe NVMe, SSD""","""15.6"", HD 1366x768, 60Hz, Non-Touch, Anti-Gla...",269,3.9
1,inspiron-15-3515,£369.00,£289.00,"""AMD Athlon™ Silver 3050U Mobile Processor wit...","""AMD® Radeon™ Graphics""","""4 GB, 1 x 4 GB, DDR4, 2400 MHz""","""128 GB, M.2, PCIe NVMe, SSD""","""15.6"", HD 1366x768, 60Hz, Non-Touch, Anti-Gla...",289,3.9
2,inspiron-15-3521,£318.99,£298.99,"""Intel® Celeron® N4020 (4 MB cache, 2 cores, 2...","""Intel® UHD Graphics 600""","""4 GB, 1 x 4 GB, DDR4, 2400 MHz""","""128 GB, M.2, PCIe NVMe, SSD""","""15.6"", HD 1366x768, 60Hz, Non-Touch, Anti-Gla...",298,4.0
3,inspiron-15-3515,£449.00,£369.00,"""AMD Ryzen™ 3 3250U Mobile Processor with Rade...","""AMD® Radeon™ Graphics""","""8 GB, 1 x 8 GB, DDR4, 2400 MHz""","""256 GB, M.2, PCIe NVMe, SSD""","""15.6"", HD 1366x768, 60Hz, Non-Touch, Anti-Gla...",369,3.9
4,inspiron-15-3521,£419.00,£369.00,"""Intel® Celeron® N4020 (4 MB cache, 2 cores, 2...","""Intel® UHD Graphics 600""","""4 GB, 1 x 4 GB, DDR4, 2400 MHz""","""256 GB, M.2, PCIe NVMe, SSD""","""15.6"", HD 1366x768, 60Hz, Non-Touch, Anti-Gla...",369,4.0
...,...,...,...,...,...,...,...,...,...,...
68,inspiron-15-7510,"£1,069.00",Nan,"""11th Generation Intel® Core™ i7-11800H (24MB ...","""NVIDIA® GeForce RTX™ 3050 Ti, 4 GB GDDR6""","""8 GB, 1 x 8 GB, DDR4, 3200 MHz""","""512 GB, M.2, PCIe NVMe, SSD""","""15.6"", FHD 1920x1080, 60Hz, Non-Touch, AG, WV...",1069,4.0
69,inspiron-16-7610,"£1,099.00",Nan,"""11th Gen Intel® Core™ i7-11800H (24 MB cache,...","""NVIDIA® GeForce RTX™ 3050, 4 GB GDDR6""","""16 GB, 2 x 8 GB, DDR4, 3200 MHz""","""512 GB, M.2, PCIe NVMe, SSD""","""16"", 3K 3072x1920, 60Hz, Non-Touch, AG, Wide ...",1099,4.1
70,inspiron-14-7420-2-in-1,"£1,099.00",Nan,"""12 Gen Intel® Core™ i7-1255U (12.0 MB cache, ...","""Intel® Iris® Xe Graphics""","""16GB, 2x8GB, DDR4, 3200MHz""","""1 TB, M.2, PCIe NVMe, SSD""","""14"", FHD+ 1920x1200, 60Hz, Touch, WVA, TrueLi...",1099,4.5
71,inspiron-16-7620-2-in-1,"£1,299.00","£1,154.00","""12th Gen Intel® Core™ i7-1260P (18 MB cache, ...","""NVIDIA® GeForce® MX550, 2 GB GDDR6""","""16 GB, 2 x 8 GB, DDR4, 3200 MHz""","""512 GB, M.2, PCIe NVMe, SSD""","""16"", FHD+ 1920x1200, 60Hz, Touch, Wide Viewin...",1154,4.3


In [5]:
# Save the information into a CSV file
df.to_csv("all_dell_inspiron_uk_laptops_specs.csv", sep=',')

time.sleep(3)
driver.close()